# Diafiltration Flowsheet Optimization Tutorial

This tutorial demonstrates how to set up, initialize, and optimize a process flowsheet using the PrOMMiS framework. We build on the example optimization script in a three-stage diafiltration process `src/prommis/uky/costing/diafiltration_flowsheet_optimization_example.py`.

**Objectives:**
- Build and initialize the diafiltration flowsheet
- Apply costing and constraints
- Run baseline simulation (before optimization)
- Perform optimization
- Compare results before and after optimization


## Step 1: Import and Setup

Import the necessary packages from [Pyomo](https://pyomo.readthedocs.io/en/stable/index.html), [IDAES](https://idaes-pse.readthedocs.io/en/stable/), and [PrOMMiS](https://prommis.readthedocs.io/en/latest/). 

The script `diafiltration.py` defines the diafiltration flowsheet: it builds the model, initializes it, and solves for membrane lengths that satisfy product specifications. In this base flowsheet, the only active constraint is the product purity requirement. A detailed walk-through of the flowsheet configuration is provided in the [tutorial on lithium and cobalt separation via diafiltration](http://localhost:8888/notebooks/diafiltration-solution.ipynb).

The script `src/prommis/uky/costing/diafiltration_flowsheet_optimization_example.py` extends this configuration by integrating the costing framework from PrOMMiS (using `QGESSCosting` in `ree_plant_capcost.py`). In later steps, this model will be extended with costing and optimization to determine stage lengths and operating parameters that minimize recovery cost while satisfying process constraints.

In [1]:
from pyomo.environ import value, units as pyunits
from idaes.core.util.model_diagnostics import degrees_of_freedom, DiagnosticsToolbox

from prommis.nanofiltration.diafiltration import (
    build_model,
    initialize_model,
    solve_model,
    unfix_opt_variables,
    add_product_constraints,
)
from prommis.uky.costing.diafiltration_flowsheet_optimization_example import (
    build_costing,
    build_optimization,
    scale_and_solve_model,
    apply_baseline_lengths,
    print_stage_cuts,
    print_io_snap,
)


## Step 2: Build and Initialize Model

In this step we construct the diafiltration flowsheet and prepare it for simulation.
Building the model creates all unit operations (stages, mixers, product outlets, etc.) and links them together through material balances. Initialization then provides reasonable starting values for the solver, ensuring that nonlinear equations converge to a feasible solution.

By following this workflow—__build__ → __initialize__ → __solve__—we ensure that the model is robust and ready for the costing and optimization steps that follow.

In [2]:
m = build_model()
dt = DiagnosticsToolbox(m)
print("Degrees of freedom:", degrees_of_freedom(m))

initialize_model(m)
solve_model(m, tee=False)
dt.assert_no_numerical_warnings()

Degrees of freedom: 0
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,1].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,2].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,3].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,4].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.stage1.permeate[0.0,5].flow_vol' to a numeric
value `0.0` outside the bounds (1e-08, None).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WAR

## Step 3: Apply Costing

In this step we apply the __QGESS-based costing framework__ from PrOMMiS. The costing model estimates capital and operating expenses for the diafiltration process, focusing on membrane modules, pumps, and associated installation and maintenance.

At this stage, __only equipment-related costs__ (capital, installation labor, and maintenance) are included. No explicit labor costs, resource consumption costs, or product sales revenues for Li or Co are considered in the baseline. As a result, the costing report shows:

- __Zero annual technical labor cost__

- __Zero administrative and support labor cost__

- __Zero sales revenue__

The costing report therefore primarily reflects the __capital expenditures (CapEx)__ for membranes and pumps and the __fixed and variable O&M__ linked to equipment performance (e.g., pumping power, pressure-drop energy). Additional elements, such as product pricing or detailed resource usage, can be layered on in later analyses if required.

In [3]:
build_costing(m)
solve_model(m, tee=False)
dt.assert_no_numerical_warnings()

## Step 4: Baseline Report (before Optimization)

With the costing framework applied, we first generate a __baseline report__ before running any optimization. This report reflects the model’s performance and cost based on the __initial guesses for decision variables__ (such as stage lengths and operating conditions).

Because the solver starts from these initial values, the baseline results can vary if users provide different initial guesses. The baseline is therefore not an optimized solution, but rather a __reference point__ that illustrates how the system performs under the default initialization. Subsequent optimization will adjust these variables to minimize the recovery cost while satisfying process constraints.

In [4]:
apply_baseline_lengths(m, L1=754, L2=758, L3=756)
solve_model(m, tee=False)  # lock in the baseline
print_io_snap(m.fs, tag="BEFORE OPTIMIZATION")
print_stage_cuts(m, label="STAGE CUTS — BEFORE OPTIMIZATION")
print(
    "Stage lengths (before):",
    [value(m.fs.stage1.length), value(m.fs.stage2.length), value(m.fs.stage3.length)],
    pyunits.get_units(m.fs.stage1.length),
)
m.fs.costing.report()


I/O SNAPSHOT: BEFORE OPTIMIZATION
[FEED  (initial; stage3.retentate_side_stream_state[0,10])]
  flow_vol: 100.0
  conc_Li:  1.7
  conc_Co:  17.0

[DIAFILTRATE (initial; mix2.inlet_1)]
  flow_vol: 30.0
  conc_Li:  0.1
  conc_Co:  0.2

[PRODUCT PERMEATE (stage3.permeate_outlet)]
  flow_vol: 113.40000000000002
  purity_Li: 0.20944272214575888
  purity_Co: 0.7905572778542411

[PRODUCT RETENTATE (stage1.retentate_outlet)]
  flow_vol: 16.60000000000002
  purity_Li: 0.00868095723195401
  purity_Co: 0.9913190427680459

[PARAMETERS]
  sieving_coefficient_Li: 1.3
  sieving_coefficient_Co: 0.5
  membrane_width (m): 1.5
  operating_pressure (psi): 145.0


STAGE CUTS — BEFORE OPTIMIZATION
Stage 1 cut (Q_perm/Q_feed): 0.872012
Stage 2 cut (Q_perm/Q_feed): 0.467132
Stage 3 cut (Q_perm/Q_feed): 0.465326

Stage lengths (before): [754, 758, 756] m

costing
------------------------------------------------------------------------------------
                                                               

Notice that in the costing report, several categories are shown as __zero__. This is expected because:

- __Labor costs__ (technical or administrative) are not included at this stage.

- __Resource costs__ (e.g., utilities beyond pumping power) are not explicitly modeled.

- __Product revenues__ from Li and Co sales are also excluded.

As a result, the __annual labor cost, administrative/support cost, and sales revenue all appear as zero__ in the baseline output.

At the same time, observe that the __total variable operating and maintenance (O&M) cost is not zero__. This comes primarily from two sources:

  __1. Membrane pressure drop__, which contributes to energy consumption.

  __2. Pumping costs__, driven by the operating pressure parameter defined in diafiltration.py.

Both of these are considered __operating costs__, and their dependence on the specified operating pressure explains why the total variable O&M cost remains non-zero even in the baseline case.

## Step 5: Optimization

The optimization problem is formulated with the objective of minimizing the cost of recovery for lithium and cobalt. To achieve this, the solver adjusts key decision variables while enforcing process constraints. Specifically, the optimization problem is subject to the following constraints:

- **Li recovery ≥ 0.945**  
- **Co recovery ≥ 0.635**  
- **Stage membrane lengths:** [0.1, 10,000] m  
- **Stage cuts:** [0.01, 0.99]  
- **Li sieving coefficient:** [0.75, 1.3]  
- **Co sieving coefficient:** [0.05, 0.5]

Through this formulation, the optimization reallocates stage lengths and adjusts separation parameters to satisfy product recovery targets while reducing overall costs.

In [5]:
build_optimization(m)
scale_and_solve_model(m)
dt.assert_no_numerical_warnings()

that are not Var, Constraint, Objective, or the model.  Skipping.


## Step 6: Final Report (After Optimization)

After the optimization completes, the model generates a __final report__ summarizing the optimized solution. This includes updated values of the key __decision variables__ (e.g., stage lengths, and stage cuts), important process parameters (such as recovery purities), and the detailed __costing breakdown__.

The final report allows direct comparison with the baseline results, showing how optimization reallocates stage lengths and adjusts operating conditions to achieve the required Li and Co recovery levels at a lower overall cost.

In [6]:
print_io_snap(m.fs, tag="AFTER OPTIMIZATION")
print_stage_cuts(m, label="STAGE CUTS — AFTER OPTIMIZATION")
print(
    "Stage lengths (after):",
    [value(m.fs.stage1.length), value(m.fs.stage2.length), value(m.fs.stage3.length)],
    pyunits.get_units(m.fs.stage1.length),
)
m.fs.costing.report()


I/O SNAPSHOT: AFTER OPTIMIZATION
[FEED  (initial; stage3.retentate_side_stream_state[0,10])]
  flow_vol: 100.0
  conc_Li:  1.7
  conc_Co:  17.0

[DIAFILTRATE (initial; mix2.inlet_1)]
  flow_vol: 30.0
  conc_Li:  0.1
  conc_Co:  0.2

[PRODUCT PERMEATE (stage3.permeate_outlet)]
  flow_vol: 119.1069038813319
  purity_Li: 0.20795597874513427
  purity_Co: 0.7920440212548656

[PRODUCT RETENTATE (stage1.retentate_outlet)]
  flow_vol: 10.893096118668112
  purity_Li: 0.008702934862727631
  purity_Co: 0.9912970651372724

[PARAMETERS]
  sieving_coefficient_Li: 1.2998743593447226
  sieving_coefficient_Co: 0.22246817852184148
  membrane_width (m): 1.5
  operating_pressure (psi): 145.0


STAGE CUTS — AFTER OPTIMIZATION
Stage 1 cut (Q_perm/Q_feed): 0.018812
Stage 2 cut (Q_perm/Q_feed): 0.874275
Stage 3 cut (Q_perm/Q_feed): 0.574837

Stage lengths (after): [1.3923442703244846, 514.6753046370288, 794.0460258755459] m

costing
----------------------------------------------------------------------------

## Key Observations

Comparison of the baseline and optimized results highlights the following:

- The optimization successfully identified a __feasible optimal solution__ within the specified constraints.

- __Stage lengths were reallocated__ across the three stages, improving the efficiency of lithium and cobalt separation.

- The __cost of recovery decreased__ from 0.054245 to 0.044076, demonstrating clear economic improvement relative to the baseline.

- Significant adjustments occurred in __membrane length, stage cuts, and the Co sieving coefficient__, reflecting how the model reallocates resources to meet recovery targets.

- The optimized solution achieved __slightly higher Li and Co purity in the permeate stream__, while also reducing overall costs.

---

## Conclusions

This case study demonstrates how the __QGESS costing framework__ can be applied to optimize a diafiltration flowsheet. The optimization reduces the cost of recovery while meeting required recovery and purity constraints, and it highlights how stage lengths and operating parameters can be reallocated to improve overall process performance.

__Next Steps:__
Future analyses could expand the costing framework by incorporating:

- __Product revenues__ from Li and Co sales,

- __Labor costs__ (technical and administrative), and

- __Resource costs__ such as utilities and consumables.

These additions would provide a more comprehensive economic assessment and enable direct comparisons between process alternatives under realistic market conditions.